# Image classification from scratch


**Description:** Training an image classifier from scratch on the Kaggle Cats vs Dogs dataset.

* Dataset: Kaggle Cats vs Dogs dataset (a binary classification problem).

* Goal: Train a neural network from scratch (not using pre-trained models).

* data loading → preprocessing → model creation → training → evaluation

## Introduction

This example shows how to do image classification from scratch, starting from JPEG
image files on disk, without leveraging pre-trained weights or a pre-made Keras
Application model. We demonstrate the workflow on the Kaggle Cats vs Dogs binary
classification dataset.

We use the `image_dataset_from_directory` utility to generate the datasets, and
we use Keras image preprocessing layers for image standardization and data augmentation.

## Setup

In [ ]:
import os
import numpy as np
import keras                                    #Build the neural network.
from keras import layers
from tensorflow import data as tf_data          #Efficient dataset pipelines.
import matplotlib.pyplot as plt

## Load the data: the Cats vs Dogs dataset

### Raw data download

First, let's download the 786M ZIP archive of the raw data:

In [ ]:


!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip




In [ ]:
# check the file is 100 % downloaded

!echo $?


# output = 0 ---- 100% downloaded
#        = 1  or 127 ----- error in download


In [ ]:
#kist the size of file



!ls -lh


In [ ]:


!unzip -q kagglecatsanddogs_5340.zip




Now we have a `PetImages` folder which contain two subfolders, `Cat` and `Dog`. Each
subfolder contains image files for each category.

In [ ]:
!ls PetImages

**Dataset Loading**

* The notebook downloads the Cats vs Dogs dataset (786 MB ZIP file) from Kaggle/Microsoft.

* It then extracts images into folders (likely cats/ and dogs/).

* After extraction, it uses keras.utils.image_dataset_from_directory to load images into TensorFlow datasets.

# **the next steps will usually be:**


---


* Splitting data → training & validation sets.

* Preprocessing → resizing, scaling pixels, augmentation.

* Model building → defining CNN layers.

* Training → fit the model with data.

* Evaluation → accuracy, loss curves, test performance.

### Filter out corrupted images



---



---



When working with lots of real-world image data, corrupted images are a common
occurence. Let's filter out badly-encoded images that do not feature the string "JFIF"
in their header.

This snippet goes through all Cat and Dog images, checks if they are valid JPEGs, and deletes the corrupted ones. Without this, your model training might crash when it encounters unreadable images.

In [ ]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")

## Generate a `Dataset`

This step loads the dataset from folders, resizes, batches, and splits it into training & validation sets — ready for preprocessing and training.

What Happens After This?







* train_ds = 80% of images (for training).

* val_ds = 20% of images (for validation).

* Both are tf.data.Dataset objects that yield (images, labels) pairs:

* images → Tensor of shape (batch_size, 180, 180, 3)

* labels → Tensor of shape (batch_size,) with values 0 (Cat) or 1 (Dog).

In [ ]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,             #Reserves 20% of images for validation set, rest for training.
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

## Visualize the data

Here are the first 9 images in the training dataset.

This block visualizes sample training images with their labels to confirm the dataset was loaded correctly.

In [ ]:

plt.figure(figsize=(10, 10))


for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
#alternative via name for pets insted 0 and 1





# Class names (order comes from folder names)
class_names = train_ds.class_names  # ["Cat", "Dog"]

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(class_names[labels[i]])  # show "Cat" or "Dog"
        plt.axis("off")


## Using image data augmentation

When you don't have a large image dataset, it's a good practice to artificially
introduce sample diversity by applying random yet realistic transformations to the
training images, such as random horizontal flipping or small random rotations. This
helps expose the model to different aspects of the training data while slowing down
overfitting.

In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images


**Why Data Augmentation?**

* Deep learning models need lots of diverse data to generalize well.

* If you train only on the original images, the model might memorize instead of learning patterns → overfitting.

* Data augmentation = generate slightly different versions of images on the fly (during training).

* This increases dataset diversity without actually collecting more images.

Let's visualize what the augmented samples look like, by applying `data_augmentation`
repeatedly to the first few images in the dataset:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")


In [ ]:
# OR do...........




#This will show 9 random augmentations of the same batch,
# e.g. cats flipped left/right, dogs slightly rotated, etc.




# Show 9 augmented images

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):   # take one batch
    augmented_images = data_augmentation(images)  # apply augmentation
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[i]).astype("uint8"))
        plt.axis("off")
    break


* Data augmentation increases dataset variety artificially.

* Helps prevent overfitting.

* In this case: horizontal flips + small rotations.

* Applied dynamically during training → each epoch may see different variations.

## Standardizing the data



---



---



Our image are already in a standard size (180x180), as they are being yielded as
contiguous `float32` batches by our dataset. However, their RGB channel values are in
the `[0, 255]` range. This is not ideal for a neural network;
in general you should seek to make your input values small. Here, we will
standardize values to be in the `[0, 1]` by using a `Rescaling` layer at the start of
our model.

**Why Standardize?**

* Images are loaded with pixel values in the range [0, 255].

* Neural networks work best when inputs are small and centered (e.g. [0,1] or [-1,1]).

* Standardization improves:
  * Numerical stability (avoids exploding gradients).
  * Training speed (optimizers converge faster).

## Two options to preprocess the data

There are two ways you could be using the `data_augmentation` preprocessor:

**Option 1: Make it part of the model**, like this:

```python
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
...  # Rest of the model
```

With this option, your data augmentation will happen *on device*, synchronously
with the rest of the model execution, meaning that it will benefit from GPU
acceleration.

Note that data augmentation is inactive at test time, so the input samples will only be
augmented during `fit()`, not when calling `evaluate()` or `predict()`.




* If you're training on GPU, this may be a good option.
* Runs on GPU together with training.
* Advantage: GPU acceleration, clean pipeline.
* Limitation: Augmentation happens synchronously, so your GPU may sometimes wait for augmentation to finish.
*⚡ Recommended if: you’re training on GPU.

---


**Option 2: apply it to the dataset**, so as to obtain a dataset that yields batches of
augmented images, like this:

```python
augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y))
```

With this option, your data augmentation will happen **on CPU**, asynchronously, and will
be buffered before going into the model.

If you're training on CPU, this is the better option, since it makes data augmentation
asynchronous and non-blocking.

In our case, we'll go with the second option. If you're not sure
which one to pick, this second option (asynchronous preprocessing) is always a solid choice.




* Applies augmentation before feeding into the model.

* Runs on CPU (separate process), and augmented images are buffered in advance.

* Advantage: asynchronous preprocessing → GPU stays busy training, no waiting.

* Limitation: Uses more RAM since preprocessed batches are buffered.

* ⚡ Recommended if: you’re training on CPU, or want more control over preprocessing.

## Configure the dataset for performance

Let's apply data augmentation to our training dataset,
and let's make sure to use buffered prefetching so we can yield data from disk without
having I/O becoming blocking:

In [ ]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

## Build a model


---



---


We'll build a small version of the Xception network. We haven't particularly tried to
optimize the architecture; if you want to do a systematic search for the best model
configuration, consider using
[KerasTuner](https://github.com/keras-team/keras-tuner).

Note that:

- We start the model with the `data_augmentation` preprocessor, followed by a
 `Rescaling` layer.
- We include a `Dropout` layer before the final classification layer.

In [ ]:


#Define the Model Function





def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Residual blocks
    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Final layers
    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    outputs = layers.Dense(units, activation=None)(x)  # logits
    return keras.Model(inputs, outputs)


In [ ]:



#Specify Input Size and Number of Classes




# Example image size
image_size = (180, 180)
num_classes = 2




In [ ]:


# Create the Model

model = make_model(input_shape=image_size + (3,), num_classes=num_classes)




In [ ]:



#Visualize the Model



keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)


In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

**Summary:**

* This is a deep CNN with residual connections, inspired by Xception-style architecture.

* It uses SeparableConv2D, BatchNorm, ReLU, GlobalAveragePooling, and Dropout.

* Suitable for binary or multi-class image classification.

## Train the model

---



---



In [ ]:

#epochs = 2 → This is just for demonstration;
#for real training, you usually use more epochs (10–50+).


epochs = 2            ## Number of times the model will go through the entire dataset

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

**Training accuracy (acc):**

* Increased from 0.63 → 0.74 → the model is learning the training data.

**Training loss:**

* Decreased from 0.64 → 0.50 → also a sign of learning.

**Validation accuracy (val_acc):**

* Stays at 0.4957 → nearly 50% → the model is not generalizing well.

**Validation loss:**

* Slightly increased → confirms overfitting or poor generalization.

**✅ Key points:**

* After training, models are saved automatically at each epoch.

* The history object contains loss and accuracy values:

We get to >90% validation accuracy after training for 25 epochs on the full dataset
(in practice, you can train for 50+ epochs before validation performance starts degrading).

## Run inference on new data


---



---


Note that data augmentation and dropout are inactive at inference time.

In [ ]:
!ls PetImages/Cat

In [ ]:
!ls PetImages/Dog

In [ ]:



img = keras.utils.load_img("PetImages/Dog/9.jpg", target_size=image_size)

#img = keras.utils.load_img("PetImages/Cat/11621.jpg", target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))



print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")



# **Improvements Over Original Code**


---


* Reusable: Can call predict_image(model, path) for any image.

* Preprocessing built-in: Resizing and normalization match your model training.

* Visualization: Automatically shows the image with prediction.

* Clear outputs: Returns both label (Cat or Dog) and probability.

* Handles single image prediction cleanly without manual tensor manipulations.

In [ ]:

# Define a reusable prediction function
def predict_image(model, img_path, image_size=(180, 180), show_image=True):
    """
    Predict whether the image is a cat or dog.

    Args:
        model: Trained Keras model
        img_path: Path to the image
        image_size: Size to resize the image (height, width)
        show_image: Whether to display the image using plt.imshow

    Returns:
        class_label: 'Cat' or 'Dog'
        probability: Probability of class 1 (Dog)
    """
    # Load and resize image
    img = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize same as training

    # Make prediction
    logits = model.predict(img_array)
    prob = float(tf.nn.sigmoid(logits[0][0]))

    # Determine class label
    class_label = "Dog" if prob > 0.5 else "Cat"

    # Show image with title
    if show_image:
        plt.imshow(img)
        plt.title(f"{class_label} ({prob*100:.2f}% Dog, {(1-prob)*100:.2f}% Cat)")
        plt.axis('off')
        plt.show()

    return class_label, prob

# Example usage
img_path = "PetImages/Dog/9.jpg"
label, probability = predict_image(model, img_path, image_size=image_size)
print(f"Prediction: {label}, Probability of Dog: {probability:.2f}")




---



---



---



---



---



---



**✨ Features**

Automatically loops over all images in a folder.

Preprocesses each image (resize + normalize) just like in training.

Predicts Cat or Dog and probability of Dog.

Optional: displays each image with title showing prediction.

Returns a Pandas DataFrame for easy analysis, export, or filtering.



```
# This is formatted as code
```



In [ ]:

import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def predict_images_in_folder(model, folder_path, image_size=(180, 180), show_images=True):
    """
    Predict all images in a folder as Cat or Dog.

    Args:
        model: Trained Keras model
        folder_path: Path to folder containing images
        image_size: Size to resize images
        show_images: Whether to display images with predictions

    Returns:
        results_df: Pandas DataFrame with columns ['filename', 'class', 'probability_dog']
    """
    results = []

    # List all image files
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)

        # Load and preprocess image
        img = keras.utils.load_img(img_path, target_size=image_size)
        img_array = keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        img_array = img_array / 255.0

        # Make prediction
        logits = model.predict(img_array)
        prob = float(tf.nn.sigmoid(logits[0][0]))
        class_label = "Dog" if prob > 0.5 else "Cat"

        # Append results
        results.append({
            'filename': img_file,
            'class': class_label,
            'probability_dog': prob
        })

        # Show image if desired
        if show_images:
            plt.imshow(img)
            plt.title(f"{class_label} ({prob*100:.2f}% Dog, {(1-prob)*100:.2f}% Cat)")
            plt.axis('off')
            plt.show()

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Example usage
folder_path = "PetImages/Cat"  # Or "PetImages/Dog"
results_df = predict_images_in_folder(model, folder_path, image_size=image_size, show_images=False)
print(results_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━